In [4]:
import numpy as np


import string
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from pyspark.ml.clustering import KMeans, KMeansModel, KMeansSummary
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, IDF, IDFModel
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, BooleanType
from pyspark.ml import Pipeline, PipelineModel

from nlp_cl_start import data_tokenizer

In [2]:
import pandas as pd

In [3]:
import pyspark as ps
spark = (ps.sql.SparkSession.builder
        .master("local[4]")
        .appName("yelp_academic")
        .getOrCreate()
        )
sc = spark.sparkContext

In [17]:
#x = PipelineModel.read()

In [24]:
spark

In [28]:
#p = x.load('tfidfkmeans/')


In [10]:
#pipe = PipelineModel.load('tfidf/')
pipe = PipelineModel.load('pipe-idf-5-7-18/')

In [6]:
#us_ex = pd.read_csv('user_from_scr_clidf18.csv')

In [12]:
#np.unique(us_ex.user_cl, return_counts=True)

(array([ 0,  1,  2,  4,  5,  7,  8,  9, 10, 11, 12, 13, 16, 17]),
 array([71, 81, 30, 47, 38, 20, 17, 22,  8, 20,  4, 10,  9,  8]))

In [5]:
from pymongo import MongoClient
import time
from bs4 import BeautifulSoup

mc = MongoClient()
db  = mc['raw_restaurants']
rv_s = db['reviews_scrap']

In [6]:
user_test = list(rv_s.find({'user_id': '_3KTfz0hcPl_yq4jOyOPiA'}))

In [7]:
u_df =pd.DataFrame(user_test)


In [8]:
df = pd.DataFrame([{'a':1, 'b':2 },{'a':9, 'b':20 },{'a':17, 'b':26 },{'a':16, 'b':2 }])
df

,a,b
0,1,2
1,9,20
2,17,26
3,16,2


In [9]:
ddd = spark.createDataFrame(df)

In [14]:
u_df.columns

Index(['_id', 'alias', 'category', 'date', 'id', 'rating', 'text', 'user_id'], dtype='object')

In [12]:
u_df2 = u_df.drop(columns=['_id', 'alias', 'category', 'date'])

In [13]:
u_df2.head()

,id,rating,text,user_id
0,ErsYo_9v8--feFCy92tymQ,5.0,Fantastic Thai food with warm service. Authent...,_3KTfz0hcPl_yq4jOyOPiA
1,W1NnSItt-SwWHrzkRtZ44g,5.0,Fantastic Vietnamese sandwich: great flavors a...,_3KTfz0hcPl_yq4jOyOPiA
2,3Q5uzROHYl1i2B_t2sIl7w,3.0,A fine dining restaurant that is in the midst ...,_3KTfz0hcPl_yq4jOyOPiA
3,-GYe-Aq7NcNJ3u-U6fmwMg,5.0,Great restaurant with an interesting mix of in...,_3KTfz0hcPl_yq4jOyOPiA
4,pPlsW81ntC2qbF0QOe6pLQ,5.0,We always have a good experience here with Ani...,_3KTfz0hcPl_yq4jOyOPiA


In [14]:
u_sp = spark.createDataFrame(u_df2)

In [15]:
from nlp_cl_start import data_tokenizer

In [17]:
u_token = data_tokenizer(u_sp)

In [18]:
u_pred = pipe.transform(u_token)

In [21]:
u_pred.columns

['id',
 'rating',
 'text',
 'user_id',
 'token',
 'vectors',
 'features',
 'prediction']

In [40]:
uu = u_pred.filter((u_pred.rating == '2.0')|(u_pred.rating == '1.0')).select( 'prediction').toPandas()

In [42]:
pd.unique(uu.prediction)

array([ 0,  1, 10])

In [43]:
from pipe_spar import  cluster_user_by_review

In [44]:
up = cluster_user_by_review(u_token, pipe)

In [46]:
%%time
up.count()

CPU times: user 19.6 ms, sys: 8.93 ms, total: 28.6 ms
Wall time: 2.12 s


6

In [49]:
a =np.array(5*[False])
a

array([False, False, False, False, False])

In [50]:
a|[False, True, False, False, False]

array([False,  True, False, False, False])